我们需要了解一下ACG的数值在各个动作中的特点，因此我们演示了以下动作，并记录了ACG数据。每个动作进行了10s。在动作间有10s的静息期。

In [167]:
import datetime
import os.path

import mne.io
import pandas as pd

import dataset as ds

In [168]:
def acc_to_locomotion2(acc):
    loco = np.linalg.norm(acc, axis=0).reshape(1, -1)
    return loco

In [169]:
from convert_x8_data import acc_to_locomotion

path = os.path.join(ds.path['free_eeg'], 'test', 'acg')
raw = mne.io.read_raw_edf(os.path.join(path, 'action_acg_2.edf'), preload=True)

loco = acc_to_locomotion(raw.get_data()[-3:, :]) / 1e6
loco2 = acc_to_locomotion2(raw.get_data()[-3:, :]) / 1e6
loco2 = acc_to_locomotion2(raw.get_data()[-3:, :]) / 1e6
raw._data[1, :] = loco
# raw._data[0, :] = loco2
raw._data[2:, :] /= 1e6

raw.rename_channels({'EEG1': 'ACG'})
# raw.rename_channels({'EEG0': 'ACG2'})

In [170]:
df_action = pd.read_excel(os.path.join(path, 'action_2.xlsx'))
df_action

In [171]:
def time_to_datetime(time):
    h = time.hour
    m = time.minute
    s = time.second
    return datetime.datetime(2024,8,3,h,m,s,tzinfo=datetime.timezone.utc)

In [172]:
import numpy as np

meas_time = raw.info["meas_date"]
df_action['开始时间'] = df_action['开始时间'].apply(lambda x: time_to_datetime(x))
onsets = (df_action['开始时间'] - meas_time).apply(lambda x: x.total_seconds())
durations = 20
descriptions = df_action['动作']
annot_action = mne.Annotations(
    onsets, durations, descriptions, orig_time=raw.info["meas_date"]
)

raw.set_annotations(raw.annotations + annot_action)

In [173]:
raw.export(raw.filenames[0].replace('.edf', '_convert.edf'), overwrite=True)

## 数据不等长问题

In [174]:
515*(8*60+35)

In [175]:
(515*(8*60+35))/500

In [176]:
(515*(8*60+35))/500 - 8*60+35